<a href="https://colab.research.google.com/github/FreddyPinto/recsys-steam-games/blob/feature/notebooks/3.1-functions-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Functions Test

El objetivo de este notebook es crear las funciones para los endpoints que se consumirán en la API. Además, se realizarán pruebas para asegurar su funcionamiento adecuado utilizando los dataset creados específicamente para este fin.

## 0 Configuraciones Globales e Importaciones

En esta sección, importamos todas las bibliotecas y/o modulos necesarios para nuestro proceso de feature engineering y establecemos configuraciones globales de ser requerido.

In [1]:
import sys
import pandas as pd

print(f"System version: {sys.version}")
print(f"Pandas version: {pd.__version__}")

System version: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
Pandas version: 1.5.3


## 1 Extracción

En esta sección, extraemos los datos de los archivos `pseudo_db1` y `pseudo_db2` en formato parquet que serán utilizados como pseudo base de datos para los endpoints.

### 1.1 Extracción de los datos

Creamos una función que lee cada archivo desde su directorio y lo carga a un DataFrame de `pandas`.

In [2]:
# Cargamos los archivos parquet
def read_parquet_files(parquet_files):
    dataframes = {}
    for name in parquet_files:
        dataframes[name] = pd.read_parquet(f'data/processed/{name}.parquet', engine='pyarrow')
    return dataframes

parquet_files = ['pseudo-db1','pseudo-db2']
dataframes = read_parquet_files(parquet_files)

# Convertimos a df.
df_pseudo_db1 = dataframes['pseudo-db1']
df_pseudo_db2 = dataframes['pseudo-db2']

## 2 Endpoints

### 2.1 Endpoint 1

def **PlayTimeGenre( *`genero` : str* )**:
    Retorna `año` con mas horas jugadas para el género dado.
Ejemplo de retorno:

``` js
{
   "Año de lanzamiento con más horas jugadas para Género X": 2013
}
```

In [84]:
def PlayTimeGenre(genre: str):
  """
  Devuelve el año con más horas jugadas para un género dado.

  Args:
    genre: El género del juego.

  Returns:
    El año de lanzamiento con más horas jugadas para un género dado.
    Ejemplo de retorno:
        {
          "Año de lanzamiento con más horas jugadas para Género X": 2013
        }
  """

  # Convierte el nombre del genero a minúsculas
  genre_lower = genre.lower()

  # Filtra el DataFrame por género
  df_genre = df_pseudo_db1[df_pseudo_db1['genres'].str.lower().str.contains(genre_lower)]

  # Verifica si el genero ingresado existe en el DataFrame
  if df_genre.empty:
        print(f"Genre {genre} not found")
        return None

  # Agrupa por año y calcula la suma total de horas jugadas para cada año
  playtime_by_year = df_genre.groupby('release_year')['playtime_forever'].sum()

  # Encuentra el año con más horas jugadas
  max_playtime_year = playtime_by_year.idxmax()
  response = {f"Año de lanzamiento con más horas jugadas para el género {genre.capitalize()}" : max_playtime_year}

  return response

In [86]:
PlayTimeGenre('Casual')

{'Año de lanzamiento con más horas jugadas para el género Casual': 2015}

In [87]:
PlayTimeGenre('CaSual')

{'Año de lanzamiento con más horas jugadas para el género Casual': 2015}

In [88]:
PlayTimeGenre('CaSuals')

Genre CaSuals not found


### 2.2 Endpoint 2

+ def **UserForGenre( *`genero` : str* )**:
    Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno:
```js
{
   "Usuario con más horas jugadas para Género X":"us213ndjss09sdf",
   "Horas jugadas":[
      {
         "Año":2013,
         "Horas":203
      },
      {
         "Año":2012,
         "Horas":100
      },
      {
         "Año":2011,
         "Horas":23
      }
   ]
}
```

In [149]:
def UserForGenre(genre: str):
  """
  Devuelve el usuario que acumula más horas jugadas para un género dado.

  Args:
    genre: El género del juego.

  Returns:
    El usuario con más horas jugadas y una lista de la acumulación de horas jugadas por año.
    Ejemplo de retorno:
        {
          "Usuario con más horas jugadas para Género X":"us213ndjss09sdf",
          "Horas jugadas":[
              {
                "Año":2013,
                "Horas":203
              },
              {
                "Año":2012,
                "Horas":100
              },
              {
                "Año":2011,
                "Horas":23
              }
          ]
        }
    """

    # Convierte el nombre del genero a minúsculas
  genre_lower = genre.lower()

  # Filtra el DataFrame por género
  df_genre = df_pseudo_db1[df_pseudo_db1['genres'].str.lower().str.contains(genre_lower)]

  # Verifica si el genero ingresado existe en el DataFrame
  if df_genre.empty:
        print(f"Genre {genre} not found")
        return None

  # Agrupa por usuario y año, y calcula la suma total de horas jugadas para cada usuario y año
  playtime_by_user_year = df_genre.groupby(['user_id', 'release_year'])['playtime_forever'].sum().reset_index()

  # Encuentra el usuario con más horas jugadas
  max_playtime_user = playtime_by_user_year.groupby('user_id')['playtime_forever'].sum().idxmax()

  # Obtiene una lista de la acumulación de horas jugadas por año para el usuario con más horas jugadas
  playtime_by_year = playtime_by_user_year[playtime_by_user_year['user_id'] == max_playtime_user][['release_year', 'playtime_forever']].rename(columns={'release_year': 'Año', 'playtime_forever': 'Horas'}).applymap(round).to_dict('records')

  response = {f"Usuario con más horas jugadas para el género {genre.capitalize()}" : max_playtime_user, "Horas jugadas": playtime_by_year}

  return response


In [150]:
UserForGenre('Action')

{'Usuario con más horas jugadas para el género Action': 'Sp3ctre',
 'Horas jugadas': [{'Año': 1995, 'Horas': 4},
  {'Año': 1997, 'Horas': 4},
  {'Año': 1999, 'Horas': 1},
  {'Año': 2000, 'Horas': 1177},
  {'Año': 2001, 'Horas': 4},
  {'Año': 2002, 'Horas': 4},
  {'Año': 2003, 'Horas': 129},
  {'Año': 2004, 'Horas': 2124},
  {'Año': 2005, 'Horas': 356},
  {'Año': 2006, 'Horas': 1504},
  {'Año': 2007, 'Horas': 1880},
  {'Año': 2008, 'Horas': 142},
  {'Año': 2009, 'Horas': 1813},
  {'Año': 2010, 'Horas': 1379},
  {'Año': 2011, 'Horas': 2591},
  {'Año': 2012, 'Horas': 6404},
  {'Año': 2013, 'Horas': 2036},
  {'Año': 2014, 'Horas': 2214},
  {'Año': 2015, 'Horas': 6572},
  {'Año': 2016, 'Horas': 503},
  {'Año': 2017, 'Horas': 722}]}

In [151]:
UserForGenre('actiOn')

{'Usuario con más horas jugadas para el género Action': 'Sp3ctre',
 'Horas jugadas': [{'Año': 1995, 'Horas': 4},
  {'Año': 1997, 'Horas': 4},
  {'Año': 1999, 'Horas': 1},
  {'Año': 2000, 'Horas': 1177},
  {'Año': 2001, 'Horas': 4},
  {'Año': 2002, 'Horas': 4},
  {'Año': 2003, 'Horas': 129},
  {'Año': 2004, 'Horas': 2124},
  {'Año': 2005, 'Horas': 356},
  {'Año': 2006, 'Horas': 1504},
  {'Año': 2007, 'Horas': 1880},
  {'Año': 2008, 'Horas': 142},
  {'Año': 2009, 'Horas': 1813},
  {'Año': 2010, 'Horas': 1379},
  {'Año': 2011, 'Horas': 2591},
  {'Año': 2012, 'Horas': 6404},
  {'Año': 2013, 'Horas': 2036},
  {'Año': 2014, 'Horas': 2214},
  {'Año': 2015, 'Horas': 6572},
  {'Año': 2016, 'Horas': 503},
  {'Año': 2017, 'Horas': 722}]}

In [152]:
UserForGenre('Ajjioni')

Genre Ajjioni not found


### 2.3 Endpoint 3


+ def **UsersRecommend( *`año` : int* )**:
   Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
  

Ejemplo de retorno:
```js
[
   {
      "Puesto 1":"X"
   },
   {
      "Puesto 2":"Y"
   },
   {
      "Puesto 3":"Z"
   }
]
```


In [97]:
def UsersRecommend(year: int):
    """
    Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado.
    (reviews.recommend = True y comentarios positivos/neutrales)

    Args:
      year: El año a filtrar.

    Returns:
      El top 3 de juegos recomendados.
      Ejemplo de retorno:
          [
            {
                "Puesto 1":"X"
            },
            {
                "Puesto 2":"Y"
            },
            {
                "Puesto 3":"Z"
            }
          ]
    """
    # Comprueba si el año se encuentra en el DataFrame
    if ~df_pseudo_db2['posted_year'].isin([year]).any():
        print(f"Year {year} not found")
        return None

    # Filtra el DataFrame por año y recomendación
    df_year = df_pseudo_db2[(df_pseudo_db2['posted_year'] == year) & (df_pseudo_db2['recommend'] == True) & (df_pseudo_db2['sentiment_analysis'] > 1)]

    # Agrupa por juego y cuenta el número de recomendaciones para cada juego
    recommendations = df_year.groupby('item_name').size()

    # Ordena los juegos en función del número de recomendaciones y toma el top 3
    top_games = recommendations.sort_values(ascending=False).head(3)

    response = [{"Puesto {}".format(i+1): game} for i, game in enumerate(top_games.index)]

    return response

In [105]:
UsersRecommend(2021)

Year 2021 not found


In [99]:
UsersRecommend(2010)

[{'Puesto 1': 'Team Fortress 2'},
 {'Puesto 2': 'Killing Floor'},
 {'Puesto 3': 'Alien Swarm'}]

### 2.4 Endpoint 4

+ def **UsersWorstDeveloper( *`año` : int* )**:
   Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
  
Ejemplo de retorno:
```js
[
   {
      "Puesto 1":"X"
   },
   {
      "Puesto 2":"Y"
   },
   {
      "Puesto 3":"Z"
   }
]
```

In [106]:
def UsersWorstDeveloper(year: int):
    """
    Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

    Args:
      year: El año a filtrar.

    Returns:
      El top 3 de desarrolladoras con juegos MENOS recomendados
      Ejemplo de retorno:
          [
            {
                "Puesto 1":"X"
            },
            {
                "Puesto 2":"Y"
            },
            {
                "Puesto 3":"Z"
            }
          ]
    """
    # Comprueba si el año se encuentra en el DataFrame
    if ~df_pseudo_db2['posted_year'].isin([year]).any():
          print(f"Year {year} not found")
          return None

    # Filtra el DataFrame por año y no recomendación
    df_year = df_pseudo_db2[(df_pseudo_db2['posted_year'] == year) & (df_pseudo_db2['recommend'] == False) & (df_pseudo_db2['sentiment_analysis'] == 0)]

    # Agrupa por juego y cuenta el número de no recomendaciones para cada juego
    not_recommendations = df_year.groupby('developer').size()

    # Ordena los juegos en función del número de no recomendaciones y toma el top 3
    top_games = not_recommendations.sort_values(ascending=False).head(3)

    response = [{"Puesto {}".format(i+1): game} for i, game in enumerate(top_games.index)]

    return response


In [107]:
UsersWorstDeveloper(2025)

Year 2025 not found


In [11]:
UsersWorstDeveloper(2014)

[{'Puesto 1': 'Valve'},
 {'Puesto 2': 'Bohemia Interactive'},
 {'Puesto 3': 'Facepunch Studios'}]

### 2.5 Endpoint 5

def **sentiment_analysis( *`empresa desarrolladora` : str* )**:
    Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total
    de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

Ejemplo de retorno:
```js
{
   "Valve":[
      {
         "Negative":1352
      },
      {
         "Neutral":2202
      },
      {
         "Positive":4840
      }
   ]
}
```


In [67]:
def sentiment_analysis(developer: str):

    """
    Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

    Args:
      developer: La desarrolladora a filtrar.

    Returns:
      El análisis de sentimiento para la desarrolladora dada.
      Ejemplo de retorno:
          {
            "Valve":[
                {
                  "Negative":1352
                },
                {
                  "Neutral":2202
                },
                {
                  "Positive":4840
                }
            ]
          }
    """
    # Convierte el nombre de la desarrolladora a minúsculas
    developer_lower = developer.lower()

    # Filtra el DataFrame por el nombre de la empresa desarrolladora
    df_dev = df_pseudo_db2[df_pseudo_db2["developer"].str.lower() == developer_lower]

    # Verifica si el desarrollador ingresado existe en el DataFrame
    if df_dev.empty:
        print(f"Developer {developer} not found")
        return None

    # Contamos el número de registros por cada categoría de sentimiento
    sentiment_counts = df_dev['sentiment_analysis'].value_counts()
    negative_reviews = sentiment_counts.get(0, 0)
    neutral_reviews = sentiment_counts.get(1, 0)
    positive_reviews = sentiment_counts.get(2, 0)

    # Crear el diccionario con el nombre de la empresa desarrolladora como llave y la lista con los conteos como valor
    response = {
        developer_lower.capitalize() : [
            {"Negative" : negative_reviews},
            {"Neutral" : neutral_reviews},
            {"Positive" : positive_reviews}
        ]
    }

    return response

In [68]:
sentiment_analysis('Valve')

{'Valve': [{'Negative': 1352}, {'Neutral': 2202}, {'Positive': 4840}]}

In [69]:
sentiment_analysis('VAlVe')

{'Valve': [{'Negative': 1352}, {'Neutral': 2202}, {'Positive': 4840}]}

In [71]:
sentiment_analysis('V54aDSA')

Developer V54aDSA not found
